In [23]:
#Importing Packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re


In [24]:
url = 'https://www.myntra.com/dresses?f=Gender%3Amen%20women%2Cwomen&sort=popularity'
driver = webdriver.Chrome()
driver.get(url)

In [25]:
title = driver.title
print(title)

Dresses Online - Low Price Offer on Dresses for Women at Myntra


In [26]:
def extract_product_info(li_element):
    try:
        wait = WebDriverWait(li_element, 10)

        # Wait for the product meta info to load
        product_meta_info = wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "product-productMetaInfo"))
        )

        product_id = li_element.get_attribute("id")

        product_name = li_element.find_element(By.CLASS_NAME, "product-product").text

        brand = li_element.find_element(By.CLASS_NAME, "product-brand").text

        try: 
            discounted_price = li_element.find_element(By.CLASS_NAME, "product-discountedPrice").text
            price = li_element.find_element(By.CLASS_NAME, "product-strike").text

        except:
            discounted_price = "NA"
            price = li_element.find_element(By.CLASS_NAME, "product-price").text

        image_element = li_element.find_element(By.TAG_NAME, "img")

        image_url = image_element.get_attribute("src")

        product_link = li_element.find_element(By.TAG_NAME, "a").get_attribute("href")

        return {
            "id": product_id,
            "product_name": product_name,
            "brand": brand,
            "price": price,
            "discounted_price": discounted_price,
            "image_url": image_url,
            "product_link": product_link,
        }

    except Exception as e:
        print(f"Error extracting product info from <li>: {e}")
        return None


In [27]:
li_container = driver.find_element(By.CLASS_NAME, "results-base")

li_elements = li_container.find_elements(By.TAG_NAME, 'li')

products = []
count = 0

for li in li_elements:
    if count > 10:
        break
    
    product = extract_product_info(li)
    if product:
        products.append(product)
    count = count + 1

print(count)
df = pd.DataFrame(products)

# Save the DataFrame to a CSV file
df.to_csv('products.csv', index=False)
print("Data saved to 'products.csv'")

11
Data saved to 'products.csv'


In [28]:
#To Extract 5 comments from the products extracted.

#Function to extract the comments from the product review pages
def extract_comments(p_id, product_review_url):
    driver.get(product_review_url)

    try:
        driver.implicitly_wait(1)
        
        review_container = driver.find_element(By.CLASS_NAME, "detailed-reviews-userReviewsContainer")
        reviews = review_container.find_elements(By.CLASS_NAME, "user-review-reviewTextWrapper")

        for i, review in enumerate(reviews[:5]):
            comment = review.text.strip()

            review_data.append({
                'product_id': p_id,
                'comment': comment
            })

    except Exception as e:
        print(f"Error scraping reviews for product ID {p_id}: {e}")

df = pd.read_csv('products.csv')

review_data = []
review_url = 'https://www.myntra.com/reviews/'

#Iterates through all the product_id in the stored csv file and creates a link form each product.
p_ids = df['id']
for p_id in p_ids:
    product_review_url = review_url + str(p_id)
    extract_comments(p_id, product_review_url)

df_reviews = pd.DataFrame(review_data)
df_reviews.to_csv('reviews.csv', index=False)

print("reviews.csv saved successfully.")

reviews.csv saved successfully.


In [29]:
driver.quit()

In [30]:
#Function to extract the ratings of each product
#To Extract 5 comments from the products extracted.

#Function to extract the comments from the product review pages
def extract_ratings(p_id, product_review_url):
    driver.get(product_review_url)

    try:
        driver.implicitly_wait(1)
        
        ratings_container = driver.find_element(By.CLASS_NAME, "detailed-reviews-ratingContainer")
        average_ratings = ratings_container.find_elements(By.CLASS_NAME, "index-flexRow index-averageRating").text
        print(average_ratings)
        # star_5 =
        # star_4 = 
        # star_3 =
        # star_2 = 
        # star_1 = 

    except Exception as e:
        print(f"Error scraping reviews for product ID {p_id}: {e}")

df = pd.read_csv('products.csv')

ratings_data = []
review_url = 'https://www.myntra.com/reviews/'

#Iterates through all the product_id in the stored csv file and creates a link form each product.
p_ids = df['id']
for p_id in p_ids:
    product_review_url = review_url + str(p_id)
    extract_ratings(p_id, product_review_url)

# df_ratings = pd.DataFrame(review_data)
# df_ratings.to_csv('reviews.csv', index=False)

# print("reviews.csv saved successfully.")


MaxRetryError: HTTPConnectionPool(host='localhost', port=58854): Max retries exceeded with url: /session/fefef101bfd0035cbd496b00adcee4e8/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000026BEF58B380>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))